In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install python-chess
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import glob
import os


In [ ]:
dataframe = pd.read_csv('/content/drive/MyDrive/chessData.csv')

In [ ]:
dataframe.head()
d=dataframe

In [ ]:
d.shape
d1=d.iloc[0:1000000 ,:]
d1.shape
d1 = d1[~d1['Evaluation'].str.contains('\ufeff|#', na=False)]
d1['Evaluation'] = d1['Evaluation'].astype(float)

<ipython-input-6-f5683cca8c9d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1['Evaluation'] = d1['Evaluation'].astype(float)


In [ ]:
import numpy as np
import chess

def generate_bitboard_with_moves(fen):
  board = chess.Board(fen)


  bitboards = np.zeros((14, 8, 8), dtype=np.int8)

  for square in chess.SQUARES:
         piece = board.piece_at(square)

         if piece is not None:

            piece_type = piece.piece_type - 1 if piece.color == chess.WHITE else piece.piece_type + 5


            bitboards[piece_type, square // 8, square % 8] = (piece_type + 1)


            # legal_moves = [move.to_square for move in board.legal_moves if move.from_square == square]


            # for move_square in legal_moves:
            #     bitboards[piece_type, move_square // 8, move_square % 8] = piece_type + 1
                # Add information about en passant square
  if board.has_legal_en_passant():
        ep_square = board.ep_square
        bitboards[12, ep_square // 8, ep_square % 8] = 1

    # Add information about whose move it is
  bitboards[13, :, :] = 1 if board.turn == chess.WHITE else 0
  return bitboards
bitboard_dataset = [generate_bitboard_with_moves(fen) for fen in d1['FEN']]

In [ ]:
# import numpy as np
# import chess

# def generate_bitboard_with_moves(fen):
#     board = chess.Board(fen)

#     # Initialize empty 3D matrix of bitboards for each piece type
#     bitboards = np.zeros((12, 8, 8), dtype=np.int8)

#     # Iterate over all squares on the board
#     for square in chess.SQUARES:
#         piece = board.piece_at(square)

#         if piece is not None:
#             # Determine piece type
#             piece_type = piece.piece_type - 1 if piece.color == chess.WHITE else piece.piece_type + 5

#             # Mark the piece's position on the bitboard
#             bitboards[piece_type, square // 8, square % 8] = (piece_type + 1)*11

#             # Generate legal moves for the piece at the square
#             legal_moves = [move.to_square for move in board.legal_moves if move.from_square == square]

#             # Mark legal moves on the bitboard
#             for move_square in legal_moves:
#                 bitboards[piece_type, move_square // 8, move_square % 8] = piece_type + 1

#     return bitboards
# bitboard_dataset = [generate_bitboard_with_moves(fen) for fen in d1['FEN']]

In [ ]:
bitboard_dataset =np.load("/content/drive/MyDrive/my_array.npy")

In [ ]:
bitboard_dataset=np.array(bitboard_dataset)
print(bitboard_dataset[0])

[[[ 0  0  0  0  0  0  0  0]
  [ 1  1  1  1  0  1  1  1]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  1  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]]

 [[ 0  2  0  0  0  0  2  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]]

 [[ 0  0  3  0  0  3  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]]

 [[ 4  0  0  0  0  0  0  4]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]]

 [[ 0  0  0  5  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0]
  [ 0  0  0 

In [ ]:
np.save('/content/drive/MyDrive/my_array.npy', bitboard_dataset)

In [ ]:
labels=d1['Evaluation']

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle



bitboard_dataset, labels = shuffle(bitboard_dataset, labels, random_state=42)


X_train, X_test, y_train, y_test = train_test_split(bitboard_dataset, labels, test_size=0.2, random_state=42)


X_train = X_train.reshape(X_train.shape[0], 14, 8, 8, 1)
X_test = X_test.reshape(X_test.shape[0], 14, 8, 8, 1)


model = keras.Sequential([
    layers.Conv3D(1, (14, 3, 3), activation='relu', input_shape=(14, 8, 8, 1)),
    layers.BatchNormalization(),

    layers.Flatten(),

    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),

    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),

    layers.Dense(1)
])

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])


model.fit(X_train, y_train, epochs=1000, batch_size=1000, validation_split=0.2)


loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Mean Absolute Error:", mae)


Epoch 1/1000
635/635 [==============================] - 9s 7ms/step - loss: 422998.3125 - mean_absolute_error: 230.5954 - val_loss: 394697.6250 - val_mean_absolute_error: 226.6120
Epoch 2/1000
635/635 [==============================] - 4s 7ms/step - loss: 391844.4688 - mean_absolute_error: 229.7552 - val_loss: 370266.8750 - val_mean_absolute_error: 228.8647
Epoch 3/1000
635/635 [==============================] - 5s 8ms/step - loss: 376491.0625 - mean_absolute_error: 234.2701 - val_loss: 364454.5312 - val_mean_absolute_error: 231.3113
Epoch 4/1000
635/635 [==============================] - 4s 7ms/step - loss: 369589.7500 - mean_absolute_error: 235.4471 - val_loss: 360375.1875 - val_mean_absolute_error: 235.4063
Epoch 5/1000
635/635 [==============================] - 4s 6ms/step - loss: 365302.2500 - mean_absolute_error: 235.4461 - val_loss: 358746.8125 - val_mean_absolute_error: 245.5010
Epoch 6/1000
635/635 [==============================] - 5s 8ms/step - loss: 361300.5312 - mean_absol

In [ ]:
model.save('my_model.keras')